# Proyecto para elegir la mejor región para la extracción de petróleo para la empresa OilyGiant.

# Contenido <a id='back'></a>

* [Introducción](#intro)
* [Etapa 1. Análisis exploratorio de datos](#data_review)
    * [Inicialización](#initialization)
    * [Cargar datos](#upload_data)
* [Etapa 2. Segmentación de conjuntos de datos](#split_data)
    * [Escalado de características](#feature_scaling)
* [Etapa 3. Evaluación de modelo](#model_evaluation)
* [Etapa 4. Preparación de calculo de ganancias](#profit_preparation)
* [Etapa 5. Calcular las ganancias de un conjunto de pozos de petróleo seleccionados y modelar las predicciones](#profits_wells_calculation)
* [Etapa 6. Calcular riesgos y ganancias para cada región](#risks_and_profits_calculation)
* [Conclusión general](#end)

# Introducción<a id='intro'></a>

Tras tener los datos sobre muestras de crudo de tres regiones, los cuales se tiene que encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo. Se conocen los parámetros de cada pozo petrolero de la región. Se plantea crear un modelo que ayude a elegir la región con el mayor margen de beneficio. Y analizar los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

# Etapa 1. Análisis exploratorio de datos<a id='data_review'></a>

## Inicialización<a id='initialization'></a>

In [1]:
#Cargar librerias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Cargar datos<a id='upload_data'></a>

In [2]:
df1 = pd.read_csv("./datasets/geo_data_0.csv")
df2 = pd.read_csv("./datasets/geo_data_1.csv")
df3 = pd.read_csv("./datasets/geo_data_2.csv")

df1.info()
print("\nDatos Duplicados: ", df1.duplicated().sum())
df1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Datos Duplicados:  0


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [26]:
df2.info()
print("\nDatos Duplicados: ", df2.duplicated().sum())
df2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Datos Duplicados:  0


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [27]:
df3.info()
print("\nDatos Duplicados: ", df3.duplicated().sum())
df3

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

Datos Duplicados:  0


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


### Conclusiones

Al estudiar los datos de los tres dfs podemos ver que se trata de una region por df donde los tres poseen el mismo numero de 5 columnas (id del cliente, f0, f1, f2 y producto(volumen de reservas en el pozo de petróleo)). 

Teniendo 100,000 filas en cada uno de los df, no se presentan datos ausentes y los tipos de datos de cada columna de cada df están correctos. De igual forma también no presenta datos duplicados en ningun df.

[Volver a Contenidos](#back)

# Etapa 2. Segmentación de conjuntos de datos<a id='split_data'></a>

In [28]:
#Creamos una funcion donde definimos las caracteristicas y el objetivo
def split_data(df):
    
    target = df['product']
    features = df.drop(["product","id"], axis=1)
    
    # Separamos los datos en conjuntos de entrenamiento y validación 
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
    
    return features_train, features_valid, target_train, target_valid

In [29]:
#Llamamos a la funcion y almacenamos los conjuntos de datos en sus respectivas variables
features_train_1, features_valid_1, target_train_1, target_valid_1 = split_data(df1)

print(features_train_1.shape)
print(features_valid_1.shape)

(75000, 3)
(25000, 3)


In [30]:
#Llamamos a la funcion y almacenamos los conjuntos de datos en sus respectivas variables
features_train_2, features_valid_2, target_train_2, target_valid_2 = split_data(df2)

print(features_train_2.shape)
print(features_valid_2.shape)

(75000, 3)
(25000, 3)


In [31]:
#Llamamos a la funcion y almacenamos los conjuntos de datos en sus respectivas variables
features_train_3, features_valid_3, target_train_3, target_valid_3 = split_data(df3)

print(features_train_3.shape)
print(features_valid_3.shape)

(75000, 3)
(25000, 3)


## Escalado de características<a id='feature_scaling'></a>

Debido a que trabajaremos con un modelo de regresión lineal escalaremos los datos para este modelo debido a que existe una diferencia de escalas entre las columnas f0, f1 y f2.

In [32]:
#Creamos una funcion para escalar las caracteristicas de los 3 dfs (3 regiones)
def scale_data(features_train, features_valid):
    #Creamos una lista de las columnas para el escalado
    numeric_cols = ['f0', 'f1', 'f2']
    #Creamos una instancia del StandardScaler
    scaler = StandardScaler()
    
    #Realizamos una copia de los datos de caracteristicas y los almacenamos como nuevas variables
    features_train_scaled = features_train.copy()
    features_valid_scaled = features_valid.copy()
    
    #Escalamos los datos con las caracteristicas de entrenamiento y transformamos los conjuntos de entrenamiento y validacion
    scaler.fit(features_train_scaled[numeric_cols])
    
    features_train_scaled[numeric_cols] = scaler.transform(features_train_scaled[numeric_cols])
    features_valid_scaled[numeric_cols] = scaler.transform(features_valid_scaled[numeric_cols])
    
    return features_train_scaled, features_valid_scaled

In [33]:
#Llamamos a la funcion y almacenamos los conjuntos de datos escalados
features_train_scaled_1, features_valid_scaled_1 = scale_data(features_train_1, features_valid_1)
features_train_scaled_2, features_valid_scaled_2 = scale_data(features_train_2, features_valid_2)
features_train_scaled_3, features_valid_scaled_3 = scale_data(features_train_3, features_valid_3)

### Conclusiones

Podemos observar como mediante la function train_test_split es posible dividir los datos de los 3 dfs en primera instancia un 75% de entrenamiento y un 25% de validacion. De igual forma mediante StandardScaler escalamos los datos para el modelo de regresion lineal debido a que existe una diferencia de escalas entre las columnas f0, f1 y f2. Tambien mencionando como desarrollar todo esto dentro de una funcion es posible llamar a esta misma para que se aplique a los 3 dfs y se pueda reutilizar el codigo

[Volver a Contenidos](#back)

# Etapa 3. Evaluación de modelo<a id='model_evaluation'></a>

In [34]:
#Creamos una funcion para entrenar el modelo y realizar predicciones
def model_training_and_predictions(features_train, features_valid, target_train, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    
    #Obtenemos el error cuadratico medio, su raiz cuadrada, el volumen medio de las predicciones y el volumen medio del objetivo
    mse = mean_squared_error(target_valid, predictions)
    rmse = mse ** 0.5
    predictions_volume_mean = predictions.mean()
    target_volume_mean = target_valid.mean()
    
    print("ECM:", mse)
    print("RECM:", rmse)
    print("Volumen Medio de las Predicciones:", predictions_volume_mean)
    print("Volumen Medio del Objetivo:", target_volume_mean)
    
    return predictions

In [35]:
valid_predictions_1 = model_training_and_predictions(features_train_scaled_1, features_valid_scaled_1, target_train_1, target_valid_1)

ECM: 1412.2129364399243
RECM: 37.5794217150813
Volumen Medio de las Predicciones: 92.59256778438035
Volumen Medio del Objetivo: 92.07859674082927


In [36]:
valid_predictions_2 = model_training_and_predictions(features_train_scaled_2, features_valid_scaled_2, target_train_2, target_valid_2)

ECM: 0.7976263360391157
RECM: 0.893099286775617
Volumen Medio de las Predicciones: 68.728546895446
Volumen Medio del Objetivo: 68.72313602435997


In [37]:
valid_predictions_3 = model_training_and_predictions(features_train_scaled_3, features_valid_scaled_3, target_train_3, target_valid_3)

ECM: 1602.377581323619
RECM: 40.02970873393434
Volumen Medio de las Predicciones: 94.96504596800489
Volumen Medio del Objetivo: 94.88423280885438


### Conclusiones

Al entrenar el modelo, realizar las predicciones y calcular las metricas en las 3 regiones en donde obtenemos lo siguiente:

* En la region 1, tenemos que el valor de error cuadratico medio es de 1412.21 y la raiz del error cuadratico medio es de 37.57, significando un error considerable el cual no se puede ignorar, aproximado de 37 mil barrilles. De igual forma vemos como el promedio del volumen de predicciones es de 92.5925 y el promedio del volumen objetivo es de 92.0785, siendo volumenes medios muy similares.

* En la region 2, tenemos que el valor de error cuadratico medio es de 0.797 y la raiz del error cuadratico medio es de 0.893, significando un error muy bajo, aproximado de menos de un barril. De igual forma vemos como el promedio del volumen de predicciones es de 68.7285 y el promedio del volumen objetivo es de 68.7231, siendo volumenes medios igual muy similares.

* En la region 3, tenemos que el valor de error cuadratico medio es de 1602.37 y la raiz del error cuadratico medio es de 40.02, significando un error considerable el cual no se puede ignorar, aproximado de 40 mil barrilles. De igual forma vemos como el promedio del volumen de predicciones es de 94.9650 y el promedio del volumen objetivo es de 94.8842, siendo volumenes medios muy similares.

En este caso, podemos observar como en la region 2 tiene el menor error posible de las tres regiones, y siendo la region 3 con el mayor error de los 3. De igual forma vemos que la region 3 tiene el mayor volumen promedio y la region 2 teniendo el menor volumne promedio de los 3

[Volver a Contenidos](#back)

# Etapa 4. Preparación de calculo de ganancias<a id='profit_preparation'></a>

Almacenamos los valores necesarios para los calculos en sus respectivas variables.

In [38]:
investment = 100000000
wells = 200
sample = 500
product_income_unit = 4500
investment_per_well = investment / wells
min_volume_unit = investment_per_well / product_income_unit
print(investment_per_well)
print(min_volume_unit)

500000.0
111.11111111111111


In [39]:
#Convertimos en Series los conjuntos de datos de las predicciones y objetivos
sample_target_1 = pd.Series(target_valid_1)
sample_predictions_1 = pd.Series (valid_predictions_1, index = target_valid_1.index)
sample_target_2 = pd.Series(target_valid_2)
sample_predictions_2 = pd.Series (valid_predictions_2, index = target_valid_2.index)
sample_target_3 = pd.Series(target_valid_3)
sample_predictions_3 = pd.Series (valid_predictions_3, index = target_valid_3.index)

### Conclusiones

Como se puede observar para preparar el calculo de ganancias potenciales que se realizara en la siguiente etapa, declaramos todos los valores necesarios para el calculo, como por ejemplo la cantidad de pozos(wells), la inversion(investment), el ingreso por pozo(investment_per_well), etc. De igual forma convertimos todos los conjuntos de datos(objetivos y predicciones) a Series.

[Volver a Contenidos](#back)

# Etapa 5. Calcular las ganancias de un conjunto de pozos de petróleo seleccionados y modelar las predicciones<a id='profits_wells_calculation'></a>

In [40]:
#Creamos una funcion para seleccionar los 200 pozos de predicciones mas altos, los alamacenamos y calculamos la ganancia potencial
def profit_calculation(target, predictions, wells, investment, product_income_unit):
    #Ordenamos las predicciones de forma descendente
    predictions_sorted = predictions.sort_values(ascending=False)
    #Sellecionamos los 200 pozos
    predictions_selected = target[predictions_sorted.index][:wells]
    #Calculamos la ganancia potencial y retornamos el resultado
    result = (product_income_unit * predictions_selected.sum()) - investment
    return result

In [41]:
#Llamamos a la funcion y almacenamos la ganacia potencial que es lo que retorna de la funcion
profit_1 = profit_calculation(sample_target_1, sample_predictions_1, wells, investment, product_income_unit)
profit_2 = profit_calculation(sample_target_2, sample_predictions_2, wells, investment, product_income_unit)
profit_3 = profit_calculation(sample_target_3, sample_predictions_3, wells, investment, product_income_unit)

In [42]:
print("Ganancias de los 200 mejores pozos petroleros de la region 1:", profit_1)
print("Ganancias de los 200 mejores pozos petroleros de la region 2:", profit_2)
print("Ganancias de los 200 mejores pozos petroleros de la region 3:", profit_3)

Ganancias de los 200 mejores pozos petroleros de la region 1: 33208260.43139851
Ganancias de los 200 mejores pozos petroleros de la region 2: 24150866.966815114
Ganancias de los 200 mejores pozos petroleros de la region 3: 27103499.635998324


### Conclusiones

Como podemos observar mediante la funcion creada en conjunto del uso de las variables declaradas en la anterior etapa se obtuvieron las ganancias potenciales de los 200 mejores pozos de cada una de las 3 regiones, de las cuales la region 1 tiene la mayor ganancia con 33,208,260.43139, siguiendo la region 3 con una ganancia de 27,103,499.63599 y finalmente la region 2 que tiene la menor ganacia de las 3 con 24,150,866.96681.

A simple vista con respecto las ganacias se podria proponer la region 1 como elegida para el desarrollo de pozos petrolíferos debido a que tiene la mayor ganacia de las 3 regiones. Sin embargo a pesar de esto seria muy apresurado el proponer la region 1 sin realizar alguna tecnica para evaluar la incertidumbre y los riesgos como el bootstrapping, sin olvidar que al realizar las predicciones para cada region se resalta que la region 1 ademas de tener la mayor ganancia tambien tuvo un RECM bastante considerable en sus predicciones a comparacion de ña region 2 que fue muy minimo el RECM. Por lo tanto antes de tomar una decision es preciso aplicar la tecnica de bootstrapping que se realizara en la siguiente etapa.

[Volver a Contenidos](#back)

# Etapa 6. Calcular riesgos y ganancias para cada región<a id='risks_and_profits_calculation'></a>

In [43]:
#Creamos una funcion para aplicar la tecnica del bootstrapping
def bootstrapping(target, predictions, wells, investment, product_income_unit):
    
    #Creamos una instancia para que el estado de random_state sea aleatorio
    state = np.random.RandomState(12345)
    values = []
    
    for i in range(1000):
        #Creamos submuestras de las predicciones y objetivo mediante bootstrapping y con esas submuestras llamamos a la funcion que calcula la ganancia potencial
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        predictions_subsample = predictions[target_subsample.index]
        profit = profit_calculation(target_subsample, predictions_subsample, wells, investment, product_income_unit)
        #Almacenamos las ganancias
        values.append(profit)

    #Convertimos la lista de valores(ganancias) en un Series
    values = pd.Series(values)
    #Obtenemos el intervalo de confianza de 95% al obtener los cuantiles de 2.5% y 97.5%
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    #Obtenemos el beneficio promedio
    mean = values.mean()
    #Obtenemos el riesgo de perdidas
    risk = (values < 0).mean()

    print(f"Intervalo de confianza del 95%: ({lower} , {upper})")
    print("Beneficio promedio:", mean)
    print("Cuantil del 97.5 %:", upper)
    print("Cuantil del 2.5 %:", lower)
    print(f"Riesgo de perdidas: {risk * 100}%")
    

In [44]:
print("Region 1:")
bootstrapping(sample_target_1, sample_predictions_1, wells, investment, product_income_unit)

Region 1:
Intervalo de confianza del 95%: (-1020900.9483793724 , 9479763.533583675)
Beneficio promedio: 4259385.269105923
Cuantil del 97.5 %: 9479763.533583675
Cuantil del 2.5 %: -1020900.9483793724
Riesgo de perdidas: 6.0%


In [45]:
print("Region 2:")
bootstrapping(sample_target_2, sample_predictions_2, wells, investment, product_income_unit)

Region 2:
Intervalo de confianza del 95%: (688732.2537050088 , 9315475.912570495)
Beneficio promedio: 5152227.734432898
Cuantil del 97.5 %: 9315475.912570495
Cuantil del 2.5 %: 688732.2537050088
Riesgo de perdidas: 1.0%


In [46]:
print("Region 3:")
bootstrapping(sample_target_3, sample_predictions_3, wells, investment, product_income_unit)

Region 3:
Intervalo de confianza del 95%: (-1288805.473297878 , 9697069.541802654)
Beneficio promedio: 4350083.627827557
Cuantil del 97.5 %: 9697069.541802654
Cuantil del 2.5 %: -1288805.473297878
Riesgo de perdidas: 6.4%


### Conclusiones

Como podemos observar tras aplicar la tecnica de bootstrapping, obtenemos el beneficio promedio, los intervalos de confianza y los riesgos de perdidas de cada una de las regiones, en donde destaca la region 2 que tiene el mas alto beneficio promedio de los 3 con 5,152,227.7344, el menor riesgo de los 3 con 1% y a diferencia de las demas regiones el limite inferior del intervalo de confianza es positivo siendo que los limites inferiores de los demas es negativo. Dejando asi la region 2 como la region mas optima para proponer a ser elegida.

Cabe destacar aunque inicialmente la region 1 parecía más atractiva en cuestion a que era ganancia potencial mas alta, la evaluación detallada de los resultados del bootstrapping confirman que la region 2 es la mejor opción, teniendo de beneficio superior y un bajo riesgo

[Volver a Contenidos](#back)

## Conclusión general <a id='end'></a>

Tras haber realizado y completado cada uno de los pasos de este proyecto desde el analisis exploratorio, la separacion de los conjuento de datos de entrenamiento y validacion de los cuales fue 75% y 25%, el entrenamiento y predicciones de un modelo de regresion lineal, los calculos para obtener la ganancia potencial de las regiones y finalmente aplicar la tecnica de bootstrapping que fue de gran ayuda para una mayor evaluacion a las regiones, obteniendo asi el beneficio promedio, los intervalos de confianza del 95% y los riesgos de perdidas de cada region. Con todo esto para poder elegir la region mas optima para abrir nuevos pozos petroleros (siendo la region 2), mencionando la creacion de funciones lo cual fue de ayuda para la reutilizacion de codigo al momento de aplicarlo para cada una de las regiones.

[Volver a Contenidos](#back)